In [1]:
import os
import gc
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.layers import Dense, Input
from collections import Counter
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K
from keras.layers import Dropout

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [ ]:
def submit(predictions):
    submit = pd.read_csv('../input/sample_submission.csv')
    submit["target"] = predictions
    submit.to_csv("submission.csv", index=False)

def fallback_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except:
        return 0.5

def auc(y_true, y_pred):
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [ ]:
df_tr = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
df = df_tr[df_tr['wheezy-copper-turtle-magic']==0]
df1 = df_test[df_test['wheezy-copper-turtle-magic']==0]

In [ ]:
NFOLDS = 5
RANDOM_STATE = 42
numeric = [c for c in df.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

In [ ]:
len_train = df.shape[0]
df1['target'] = -1
data = pd.concat([df, df1])
data['magic_count'] = data.groupby(['wheezy-copper-turtle-magic'])['id'].transform('count')
data = pd.concat([data, pd.get_dummies(data['wheezy-copper-turtle-magic'])], axis=1, sort=False)

df = data[:len_train]
df1 = data[len_train:]

In [ ]:
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=RANDOM_STATE)

In [2]:
gc.collect()

20

In [ ]:
y = df.target
ids = df.id.values
train = df.drop(['id', 'target'], axis=1)
test_ids = df1.id.values
test = df1[train.columns]

oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))

scl = preprocessing.StandardScaler()
scl.fit(pd.concat([train, test]))
train = scl.transform(train)
test = scl.transform(test)


X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state = 42)

In [ ]:


inp = Input(shape=(X_train.shape[1],))
x = Dense(2000, activation="relu")(inp)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(1000, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(500, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(100, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
out = Dense(1, activation="sigmoid")(x)
    
clf = Model(inputs=inp, outputs=out)
clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
clf.summary()
#     es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=10,
#                                  verbose=1, mode='max', baseline=None, restore_best_weights=True)

#     rlr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.5,
#                                       patience=3, min_lr=1e-6, mode='max', verbose=1)
#, callbacks=[es, rlr]
clf.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)
    
# val_preds = clf.predict(val_x)
# test_fold_preds = clf.predict(test)
    
# print("AUC = {}".format(metrics.roc_auc_score(val_y, val_preds)))
# oof_preds[val_] = val_preds.ravel()
# test_preds += test_fold_preds.ravel() / NFOLDS
    
K.clear_session()
gc.collect()